In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import backend
import tensorflow
tensorflow.set_random_seed(7)
from sklearn.model_selection import train_test_split
import numpy
numpy.random.seed(7)

Using TensorFlow backend.


## Restricting GPU memory usage

The code here should be added to any work you do on Volta.  If you don't, then your code will monopolize all available memory on each of the 4 GPUs on the machine, preventing others from working on it.  If you do **that**, you will be frowned upon.

The code in the next cell has the effect that:
1. Memory use will start off with some small fraction of the memory on each GPU.
1. It will grow if necessary (since `allow_growth` is set to `True`).
1. It will max out at 5% of overall memory.  Given the GPUs we have, this gives you (4 x 808 MB), which should be sufficient here.

In [2]:
########################
# Limit TensorFlow GPU use.
config = tensorflow.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.05
backend.tensorflow_backend.set_session(tensorflow.Session(config=config))
########################

## Loading our data

The function `train_test_split` comes from `sklearn` (from `SciKit`). Two things to note:
1. Setting `test_size` to `0.25` means that 25% of the data is held aside as testing data.  
1. Randomness here can affect overall results.

In [18]:
dataset = numpy.loadtxt("diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]

train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.25)

## Building and evaluating our model

The initial model is a simple network, with two hidden layers, each of which uses the RELU function for activation.  There is also a simple single-class output neuron using the Sigmoid logistic function.  The loss function is binary cross entropy (and so we want to measure the associated binary accuracy value).  The model is trained using a batch size equal to all of the training data, for 1,000 epochs.

For this network, do the following:
1. The Crosse Entropy:  
What is entropy? The value of Entropy indicates how much effort we should do to represent the information of a uncerten thing. For example, if we konw the real probility for this thing, we can calculate the entropy by using the formula:
$$H(p) = \sum_{i}p_i\log_2\frac{1}{p_i}$$In this formula, we assume that we know the precise value of each $p_i$. But if we don't know the real probability clearly, we can assign a wrong (or predict a) probability for this distribution, marked as $q_i$. So, this is the crosse entrop and the formula is:
$$H(p,q) = \sum_{i}p_i\log_2\frac{1}{q_i}$$The value of crosse entropy is always bigger than the entropy because the $p_i$ is not equals to $q_i$ and we can proof that by using mathematical approaches. The more close H(p) and H(p,q), the more accuracy we get.  
Go back to the deep learning area, we build the model and train the model, trying to get more accuracy outputs. We use loss function to indicate how far we can reach the goal(because we know the output and input for all the train dataset, in surpervised learning area). The less loss we get, the more accuracy the output is. The crosse entropy is the one of the loss funcitons. So, for each epoch, we try to decrease the crosse entropy to increase the accuracy of the model for those train dataset.  
In keras framework, the binary crosse entropy is a kind of specific case of catogory crosse entropy and it is also called sigmoid crosse entropy. Because the output of the dataset is binary(one hot vector), so we chose the binary crosse entropy as the loss function. If the output is not one hot such as our homework assignment 3, we should chosse the categorical corsse entropy as the loss function. 
   
  
1. The table:  
<table>
  <thead>
    <tr>
      <th>Epoch</th>
      <th>1000</th>
      <th>2000</th>
      <th>5000</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>acc_train_set</td>
      <td>77.26%</td>
      <td>86.98%</td>
      <td>89.76%</td>
    </tr>
    <tr>
      <td>acc_test_set</td>
      <td>72.40%</td>
      <td>77.60%</td>
      <td>68.25%</td>
    </tr>
  </tbody>
</table>  
when we train it over and over , the accuracy of the training data increase but the accuracy of testing data decrest, which means the model is overfitting. The reason is when we train the model over and over again(increase the Epoch), the model become more precise to the training dataset but it can't work well in another dataset.
  
1. About the dropout layer.  
Dropout is the rate(0,1) indicate how many random neurons we should igonre from this layer to the next layer when we train the data.
If we add a dropout rate between the hidden layers, I found that the accuracy of testing data might be larger than the accuracy in train dataset, which means the dropout layer can fix the overfitting problems in CNN.  
The idea is to ignore some neurons to make the weights updating more randomly.


In [116]:
# create model
model = Sequential()
model.add(Dense(25, input_dim=8, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(25, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
# Fit the model
model.fit(train_x, train_y, epochs=1000, batch_size=len(train_x), validation_data=(test_x, test_y), verbose=1)

Train on 576 samples, validate on 192 samples
Epoch 1/1000
576/576 [==============================] - 2s 3ms/step - loss: 6.1456 - binary_accuracy: 0.4670 - val_loss: 4.9831 - val_binary_accuracy: 0.4531
Epoch 2/1000
576/576 [==============================] - 0s 23us/step - loss: 6.1364 - binary_accuracy: 0.4583 - val_loss: 4.7494 - val_binary_accuracy: 0.4583
Epoch 3/1000
576/576 [==============================] - 0s 20us/step - loss: 5.8706 - binary_accuracy: 0.5000 - val_loss: 4.5126 - val_binary_accuracy: 0.4948
Epoch 4/1000
576/576 [==============================] - 0s 18us/step - loss: 5.8919 - binary_accuracy: 0.4792 - val_loss: 4.2773 - val_binary_accuracy: 0.5156
Epoch 5/1000
576/576 [==============================] - 0s 26us/step - loss: 5.4016 - binary_accuracy: 0.5052 - val_loss: 4.0186 - val_binary_accuracy: 0.5156
Epoch 6/1000
576/576 [==============================] - 0s 23us/step - loss: 5.5020 - binary_accuracy: 0.5174 - val_loss: 3.7954 - val_binary_accuracy: 0.5625
E

Epoch 52/1000
576/576 [==============================] - 0s 20us/step - loss: 3.6891 - binary_accuracy: 0.5573 - val_loss: 1.3198 - val_binary_accuracy: 0.6771
Epoch 53/1000
576/576 [==============================] - 0s 22us/step - loss: 3.6163 - binary_accuracy: 0.5521 - val_loss: 1.3418 - val_binary_accuracy: 0.6667
Epoch 54/1000
576/576 [==============================] - 0s 19us/step - loss: 3.5346 - binary_accuracy: 0.5590 - val_loss: 1.3902 - val_binary_accuracy: 0.6823
Epoch 55/1000
576/576 [==============================] - 0s 19us/step - loss: 3.4650 - binary_accuracy: 0.5556 - val_loss: 1.4258 - val_binary_accuracy: 0.6771
Epoch 56/1000
576/576 [==============================] - 0s 19us/step - loss: 3.7243 - binary_accuracy: 0.5538 - val_loss: 1.4680 - val_binary_accuracy: 0.6719
Epoch 57/1000
576/576 [==============================] - 0s 19us/step - loss: 3.6770 - binary_accuracy: 0.5243 - val_loss: 1.4901 - val_binary_accuracy: 0.6562
Epoch 58/1000
576/576 [=================

Epoch 104/1000
576/576 [==============================] - 0s 20us/step - loss: 1.6576 - binary_accuracy: 0.5625 - val_loss: 0.7541 - val_binary_accuracy: 0.6615
Epoch 105/1000
576/576 [==============================] - 0s 16us/step - loss: 1.6971 - binary_accuracy: 0.5747 - val_loss: 0.7438 - val_binary_accuracy: 0.6562
Epoch 106/1000
576/576 [==============================] - 0s 23us/step - loss: 1.6950 - binary_accuracy: 0.5677 - val_loss: 0.7332 - val_binary_accuracy: 0.6562
Epoch 107/1000
576/576 [==============================] - 0s 21us/step - loss: 1.7041 - binary_accuracy: 0.5677 - val_loss: 0.7220 - val_binary_accuracy: 0.6562
Epoch 108/1000
576/576 [==============================] - 0s 22us/step - loss: 1.7070 - binary_accuracy: 0.5503 - val_loss: 0.7097 - val_binary_accuracy: 0.6719
Epoch 109/1000
576/576 [==============================] - 0s 20us/step - loss: 1.6524 - binary_accuracy: 0.5521 - val_loss: 0.6981 - val_binary_accuracy: 0.6771
Epoch 110/1000
576/576 [==========

Epoch 155/1000
576/576 [==============================] - 0s 16us/step - loss: 0.9648 - binary_accuracy: 0.5990 - val_loss: 0.6135 - val_binary_accuracy: 0.7031
Epoch 156/1000
576/576 [==============================] - 0s 17us/step - loss: 1.0977 - binary_accuracy: 0.5816 - val_loss: 0.6124 - val_binary_accuracy: 0.7135
Epoch 157/1000
576/576 [==============================] - 0s 17us/step - loss: 0.8782 - binary_accuracy: 0.5833 - val_loss: 0.6117 - val_binary_accuracy: 0.7135
Epoch 158/1000
576/576 [==============================] - 0s 17us/step - loss: 0.9703 - binary_accuracy: 0.5851 - val_loss: 0.6115 - val_binary_accuracy: 0.7083
Epoch 159/1000
576/576 [==============================] - 0s 20us/step - loss: 0.9746 - binary_accuracy: 0.5660 - val_loss: 0.6117 - val_binary_accuracy: 0.6979
Epoch 160/1000
576/576 [==============================] - 0s 18us/step - loss: 1.0296 - binary_accuracy: 0.5729 - val_loss: 0.6127 - val_binary_accuracy: 0.6979
Epoch 161/1000
576/576 [==========

Epoch 206/1000
576/576 [==============================] - 0s 20us/step - loss: 0.6996 - binary_accuracy: 0.6215 - val_loss: 0.6195 - val_binary_accuracy: 0.6771
Epoch 207/1000
576/576 [==============================] - 0s 19us/step - loss: 0.7514 - binary_accuracy: 0.6406 - val_loss: 0.6202 - val_binary_accuracy: 0.6771
Epoch 208/1000
576/576 [==============================] - 0s 19us/step - loss: 0.7809 - binary_accuracy: 0.6181 - val_loss: 0.6207 - val_binary_accuracy: 0.6771
Epoch 209/1000
576/576 [==============================] - 0s 21us/step - loss: 0.7225 - binary_accuracy: 0.6528 - val_loss: 0.6205 - val_binary_accuracy: 0.6771
Epoch 210/1000
576/576 [==============================] - 0s 19us/step - loss: 0.8634 - binary_accuracy: 0.5972 - val_loss: 0.6203 - val_binary_accuracy: 0.6823
Epoch 211/1000
576/576 [==============================] - 0s 25us/step - loss: 0.7812 - binary_accuracy: 0.6458 - val_loss: 0.6200 - val_binary_accuracy: 0.6771
Epoch 212/1000
576/576 [==========

...

Epoch 767/1000
576/576 [==============================] - 0s 18us/step - loss: 0.5794 - binary_accuracy: 0.6823 - val_loss: 0.5758 - val_binary_accuracy: 0.7448
Epoch 768/1000
576/576 [==============================] - 0s 19us/step - loss: 0.5723 - binary_accuracy: 0.7222 - val_loss: 0.5758 - val_binary_accuracy: 0.7396
Epoch 769/1000
576/576 [==============================] - 0s 17us/step - loss: 0.5719 - binary_accuracy: 0.7014 - val_loss: 0.5758 - val_binary_accuracy: 0.7396
Epoch 770/1000
576/576 [==============================] - 0s 18us/step - loss: 0.5896 - binary_accuracy: 0.6719 - val_loss: 0.5758 - val_binary_accuracy: 0.7448
Epoch 771/1000
576/576 [==============================] - 0s 19us/step - loss: 0.5582 - binary_accuracy: 0.7066 - val_loss: 0.5759 - val_binary_accuracy: 0.7500
Epoch 772/1000
576/576 [==============================] - 0s 21us/step - loss: 0.5909 - binary_accuracy: 0.6944 - val_loss: 0.5759 - val_binary_accuracy: 0.7500
Epoch 773/1000
576/576 [==========

Epoch 818/1000
576/576 [==============================] - 0s 20us/step - loss: 0.5856 - binary_accuracy: 0.7014 - val_loss: 0.5753 - val_binary_accuracy: 0.7448
Epoch 819/1000
576/576 [==============================] - 0s 17us/step - loss: 0.5763 - binary_accuracy: 0.6979 - val_loss: 0.5753 - val_binary_accuracy: 0.7448
Epoch 820/1000
576/576 [==============================] - 0s 22us/step - loss: 0.6006 - binary_accuracy: 0.6962 - val_loss: 0.5755 - val_binary_accuracy: 0.7448
Epoch 821/1000
576/576 [==============================] - 0s 19us/step - loss: 0.5603 - binary_accuracy: 0.6892 - val_loss: 0.5758 - val_binary_accuracy: 0.7448
Epoch 822/1000
576/576 [==============================] - 0s 23us/step - loss: 0.5629 - binary_accuracy: 0.6997 - val_loss: 0.5760 - val_binary_accuracy: 0.7500
Epoch 823/1000
576/576 [==============================] - 0s 17us/step - loss: 0.5779 - binary_accuracy: 0.7205 - val_loss: 0.5763 - val_binary_accuracy: 0.7500
Epoch 824/1000
576/576 [==========

Epoch 869/1000
576/576 [==============================] - 0s 19us/step - loss: 0.5580 - binary_accuracy: 0.6927 - val_loss: 0.5723 - val_binary_accuracy: 0.7500
Epoch 870/1000
576/576 [==============================] - 0s 17us/step - loss: 0.5662 - binary_accuracy: 0.7135 - val_loss: 0.5723 - val_binary_accuracy: 0.7500
Epoch 871/1000
576/576 [==============================] - 0s 22us/step - loss: 0.5710 - binary_accuracy: 0.7014 - val_loss: 0.5724 - val_binary_accuracy: 0.7552
Epoch 872/1000
576/576 [==============================] - 0s 19us/step - loss: 0.5759 - binary_accuracy: 0.7031 - val_loss: 0.5726 - val_binary_accuracy: 0.7552
Epoch 873/1000
576/576 [==============================] - 0s 19us/step - loss: 0.5783 - binary_accuracy: 0.6701 - val_loss: 0.5727 - val_binary_accuracy: 0.7552
Epoch 874/1000
576/576 [==============================] - 0s 21us/step - loss: 0.5519 - binary_accuracy: 0.7153 - val_loss: 0.5727 - val_binary_accuracy: 0.7552
Epoch 875/1000
576/576 [==========

Epoch 920/1000
576/576 [==============================] - 0s 20us/step - loss: 0.5464 - binary_accuracy: 0.7240 - val_loss: 0.5719 - val_binary_accuracy: 0.7552
Epoch 921/1000
576/576 [==============================] - 0s 18us/step - loss: 0.5584 - binary_accuracy: 0.7222 - val_loss: 0.5719 - val_binary_accuracy: 0.7552
Epoch 922/1000
576/576 [==============================] - 0s 20us/step - loss: 0.5733 - binary_accuracy: 0.6892 - val_loss: 0.5720 - val_binary_accuracy: 0.7604
Epoch 923/1000
576/576 [==============================] - 0s 22us/step - loss: 0.5564 - binary_accuracy: 0.6979 - val_loss: 0.5720 - val_binary_accuracy: 0.7552
Epoch 924/1000
576/576 [==============================] - 0s 37us/step - loss: 0.5697 - binary_accuracy: 0.6927 - val_loss: 0.5721 - val_binary_accuracy: 0.7552
Epoch 925/1000
576/576 [==============================] - 0s 20us/step - loss: 0.5655 - binary_accuracy: 0.7240 - val_loss: 0.5722 - val_binary_accuracy: 0.7552
Epoch 926/1000
576/576 [==========

Epoch 971/1000
576/576 [==============================] - 0s 23us/step - loss: 0.5646 - binary_accuracy: 0.7014 - val_loss: 0.5724 - val_binary_accuracy: 0.7552
Epoch 972/1000
576/576 [==============================] - 0s 19us/step - loss: 0.5771 - binary_accuracy: 0.7153 - val_loss: 0.5724 - val_binary_accuracy: 0.7552
Epoch 973/1000
576/576 [==============================] - 0s 18us/step - loss: 0.5498 - binary_accuracy: 0.6927 - val_loss: 0.5724 - val_binary_accuracy: 0.7552
Epoch 974/1000
576/576 [==============================] - 0s 18us/step - loss: 0.5636 - binary_accuracy: 0.6927 - val_loss: 0.5724 - val_binary_accuracy: 0.7552
Epoch 975/1000
576/576 [==============================] - 0s 20us/step - loss: 0.5668 - binary_accuracy: 0.6962 - val_loss: 0.5725 - val_binary_accuracy: 0.7552
Epoch 976/1000
576/576 [==============================] - 0s 21us/step - loss: 0.5437 - binary_accuracy: 0.7205 - val_loss: 0.5726 - val_binary_accuracy: 0.7500
Epoch 977/1000
576/576 [==========

## Evaluation of final model on training data

In [117]:
# evaluate the model
scores = model.evaluate(train_x, train_y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

576/576 [==============================] - 0s 112us/step

binary_accuracy: 75.52%


## Evaluation of final model on testing data

In [118]:
# evaluate the model
scores = model.evaluate(test_x, test_y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

192/192 [==============================] - 0s 131us/step

binary_accuracy: 75.00%
